# Text Mining Grouo Project
## DE-EN Corpus

##### TOC for Implemented Metrics.

---

In [ ]:
# Necessary Installs.
# !pip install rouge

In [18]:
# Imports
import pandas as pd
from collections import Counter
from rouge import Rouge

Load Dataset

In [4]:
df1 = pd.read_csv("scores.csv")

In [5]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21704 entries, 0 to 21703
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   source       21704 non-null  object 
 1   reference    21704 non-null  object 
 2   translation  21704 non-null  object 
 3   z-score      21704 non-null  float64
 4   avg-score    21704 non-null  float64
 5   annotators   21704 non-null  int64  
dtypes: float64(2), int64(1), object(3)
memory usage: 1017.5+ KB


In [6]:
df1.head()

,source,reference,translation,z-score,avg-score,annotators
0,"Ihr Zeitlupentempo maßen sie, als sie vor Spit...",Her timeless pace measures them when they equi...,Their slow speed was measured by researchers o...,-0.345024,76.0,1
1,"Er sagte, dass die Bereiche ruhige Treffpunkte...",He said the areas offer quiet meeting points b...,He said the spaces provided calm meeting point...,0.903800,97.5,2
2,Für die Geschäftsleute an der B 27 ist es nur ...,"For businessmen at the B 27, it's only a small...",This is only a small consolation for businesse...,0.700503,94.0,1
3,Diese Fähigkeit sei möglicherweise angeboren o...,This ability may be born or developed with gen...,"This ability may be innate, or may develop as ...",-1.256572,51.5,2
4,Weil sie Wassertemperaturen um die sechs Grad ...,Because they prefer water temperatures around ...,They generally only come to the surface in win...,0.293909,87.0,2


---
PreProcessing

In [11]:
# Pre processing of this set.
df = df1.copy()

for x in ["source","reference","translation"]:
    # lowercase.
    df[x] = df1[x].str.lower()

---
Metrics

Rouge metric as described in

https://towardsdatascience.com/the-ultimate-performance-metric-in-nlp-111df6c64460

In [20]:
reference = df["reference"][0]
model_out = df["translation"][0]

In [21]:
rouge = Rouge()

In [24]:
print(reference)
print(model_out)

her timeless pace measures them when they equipped six animals with a broadcaster before spitsbergen.
their slow speed was measured by researchers off svalbard, who fitted six animals with a tracker.


In [22]:
# The get scores method returns three metrics, F1 score, p precision and recall r.
# For each unigram,bigram and Longest sequence.
rouge.get_scores(model_out,reference)

[{'rouge-1': {'f': 0.2580645111342353, 'p': 0.25, 'r': 0.26666666666666666},
  'rouge-2': {'f': 0.20689654673008337, 'p': 0.2, 'r': 0.21428571428571427},
  'rouge-l': {'f': 0.2580645111342353, 'p': 0.25, 'r': 0.26666666666666666}}]

In [59]:
# For the entire model, model_out and reference need to be lists of strings.
model_out = df["translation"].to_list()
reference = df["reference"].to_list()
rouge_scores = rouge.get_scores(model_out,reference)

In [60]:
# For each of the three scores, output a new column in the df with the f scores.
for key in rouge_scores[0].keys():
    df[(key+" score")] = pd.Series([score[key]["f"] for score in rouge_scores])

---
### Comparison of Applied Metrics
Because the numeric system used for all of these can be different, the best way to compare them is by checking the correlation with the annotator's scores.

In [78]:
# Thankfully, Pandas has a corr method. :)
print(df.iloc[:,-3].corr(df.loc[:,"avg-score"]))
print(df.iloc[:,-2].corr(df.loc[:,"avg-score"]))
print(df.iloc[:,-1].corr(df.loc[:,"avg-score"]))

0.27712388715157904
0.25796024900539144
0.2858468104864439


In [76]:
## Beautiful.